# Simple Error Model for Twinkles

This notebook will calculate a simple error model for the Twinkles data as a function of the seeing and depth.

### Requirements

You will need the DESC `Monitor` and its dependencies.

You will also need the truth database for the desired Twinkles run.

In [ ]:
import os
import desc.monitor
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload 2

### Get differences between true and observed fluxes

We first want to get all the differences for the run between the true and observed fluxes and store them in a Pandas Dataframe.

In [ ]:
star_db_name = 'twinkles_run1.1.db'

In [ ]:
truth_dbConn = desc.monitor.truthDBInterface(database=star_db_name, driver='sqlite')
dbConn_twinkles = desc.monitor.dbInterface(database='DESC_Twinkles_Level_2',
                                           host='127.0.0.1', port='3307',
                                           driver='mysql')

In [ ]:
worker = desc.monitor.Monitor(dbConn_twinkles, truth_dbConn=truth_dbConn)

In [ ]:
depth_curve = worker.measure_depth_curve()

In [ ]:
seeing_curve = worker.measure_seeing_curve()

In [ ]:
worker.calc_flux_residuals(with_depth_curve=depth_curve, with_seeing_curve=seeing_curve, for_visits=depth_curve.lightcurve['obsHistId'][:50])

### Plotting Bias and Sigma

Now that we have the data we need in the form of mean flux residuals and mean squared flux residuals for each visit we can combine this with our depth and seeing information to construct plots that show the bias and sigma as functions of these values.

In [ ]:
fig = worker.plot_bias_map(with_bins=6, in_band='r')

In [ ]:
fig = worker.plot_bias_scatter(in_band='r')

In [ ]:
fig = worker.plot_variance_map(with_bins=6, in_band='r')

In [ ]:
fig = worker.plot_variance_scatter(in_band='r')